In [71]:
from bs4 import BeautifulSoup
import requests

from fake_useragent import FakeUserAgent

from dotenv import dotenv_values

import re
import csv
import os.path

import pandas as pd

In [72]:
# Подгрузка данных из файла окружения
config = dotenv_values(".env")

# Обращения
# config[hh_api_name]
# config[hh_api_Client_ID]
# config[hh_api_Client_Secret]

In [92]:
# Основные регулярные выражения для проекта
re_vacancy_id_hh = r'\/vacancy\/(\d+)\?'
re_vacancy_id_rabota = r'\/vacancy\/(\d+)'
re_vacancy_id_finder = r'\/vacancies\/(\d+)'
re_vacancy_id_zarplata = r'\/vacancy\/card\/id(\d+)'

# re.search(re_vacancy_id, string).group(1)

In [ ]:
class Rabota1000_Parser:
    # Класс для парсинга вакансий с ресурса Rabota1000.ru
    

    def __init__(self, city:str='russia'):
        self.max_page_count = 5
        self.basic_url = 'https://rabota1000.ru/russia/'
        self.vac_name_list = []
        self.vac_name_list = self.get_vac_name_into_file('vacancy_name_to_pars.txt')
        
        ua = FakeUserAgent()
        headers = {'user-agent':ua.random}

        self._pars()

    def _pars(self):
        if not os.path.exists('pars_link.csv'):
            with open('pars_link.csv', 'w', newline='', encoding='utf-8') as csv_file:
                names = ['vac_name', 'link', 'source', 'vac_id']
                file_writer = csv.DictWriter(csv_file, delimiter = ",", lineterminator="\r", fieldnames=names)
                file_writer.writeheader()

            for vac_name in self.vac_name_list:
                print(vac_name)
                try:
                    used_url = self.basic_url + vac_name + "/"
                    response = requests.get(used_url)
                    response.raise_for_status()
                    for i in range(1, self.max_page_count+1):
                        print(i, end=' ')
                        used_url = f'{self.basic_url}{vac_name}?p={i}'
                        page = requests.get(used_url)
                        soup = BeautifulSoup(page.text, 'html.parser')

                        # 20 ссылок на одной странице
                        links = [requests.get(link['href']).url for link in soup.findAll('a', attrs={'@click':'vacancyLinkClickHandler'})]
                        sources = [source.text for source in soup.findAll('span', attrs={'class':'text-sky-600'})]

                        links_to_save = [[vac_name, link, source] for link, source in zip(links, sources)]

                        with open('pars_link.csv', 'a', newline='', encoding='utf-8') as csv_file:
                            writer = csv.writer(csv_file)
                            writer.writerows(links_to_save)
                            
                except HTTPError as exc:
                    code = exc.response.status_code
                    print(code)
                print()

        links_for_processing = []
        with open('pars_link.csv', 'r', encoding='utf-8') as csv_file:
            reader = csv.reader(csv_file)
            labels = next(reader, None)
            for row in reader:
                links_for_processing.append(dict(zip(labels, row)))
        
        for item in links_for_processing:
            if item['source'] == 'hh.ru':
                item['vac_id'] = re.search(re_vacancy_id_hh, item['link']).group(1)
            elif item['source'] == 'finder.vc':
                item['vac_id'] = re.search(re_vacancy_id_finder, item['link']).group(1)
            elif item['source'] == 'zarplata.ru':
                item['vac_id'] = re.search(re_vacancy_id_zarplata, item['link']).group(1)
            else:
                item['vac_id'] = re.search(re_vacancy_id, item['link']).group(1)

        pre_resualt = []
        for link in links_for_processing:
            if link['source'] == 'hh.ru':
                pre_resualt.append(self._pars_url_hh(link['vac_id']))
            elif link['source'] == 'zarplata.ru':
                pre_resualt.append(self._pars_url_zarplata(link['vac_id']))
            else:
                pre_resualt.append(self._pars_url_other(link['vac_id']))

    def get_vac_name_into_file(self, vac_file_path:str)->list:
        vac_name_list = []
        with open(vac_file_path, mode='r', encoding="utf-8") as file_vac:
            vac_name_list = list(map(lambda x: x.lower().replace('\n', '').replace(' ','+'), file_vac.readlines()))

        return vac_name_list

    def _pars_url_hh(self, id)->dict:
        res = {}
        data = requests.get(f'https://api.hh.ru/vacancies/{id}').json()
        res['vac_link'] = f'https://hh.ru/vacancy/{id}' # Ссылка
        res['name'] = data['name']                      # Название
        res['city'] = data['area']['name']              # Город
        res['company'] = data['employer']['name']       # Назвнание компании публикующей вакансию
        res['experience'] = data['experience']['name']  # Опыт работы (нет замены на jun mid и sin)
        res['schedule'] = data['schedule']['name']      # Тип работы (офис/удаленка и тд)
        res['employment'] = data['employment']['name']  # График работы
        res['skills'] = data['key_skills']              # Ключевые навыки
        res['description'] = data['description']        # Полное описание (html теги не убраны)
        if data['salary'] == None: 
            res['salary'] = 'Договорная'                # Если ЗП не указано то пишем договорная
        else:
            res['salary'] = data['salary']              # Если есть то берем как есть
        res['time'] = data['published_at']              # Дата и время публикации

        return res


    def _pars_url_zarplata(self, id)->dict:
        res = {}
        data = requests.get(f'https://api.zarplata.ru/vacancies/{id}').json()
        res['vac_link'] = f'https://www.zarplata.ru/vacancy/card/id{id}' # Ссылка
        res['name'] = data['name']                      # Название
        res['city'] = data['area']['name']              # Город
        res['company'] = data['employer']['name']       # Назвнание компании публикующей вакансию
        res['experience'] = data['experience']['name']  # Опыт работы (нет замены на jun mid и sin)
        res['schedule'] = data['schedule']['name']      # Тип работы (офис/удаленка и тд)
        res['employment'] = data['employment']['name']  # График работы
        res['skills'] = data['key_skills']              # Ключевые навыки
        res['description'] = data['description']        # Полное описание (html теги не убраны)
        if data['salary'] == None: 
            res['salary'] = 'Договорная'                # Если ЗП не указано то пишем договорная
        else:
            res['salary'] = data['salary']              # Если есть то берем как есть
        res['time'] = data['published_at']

        return res


    def _pars_url_other(self, id)->dict:
        pass

    

parser = Rabota1000_Parser()

Необходимо достать в результате

- Название вакансии +
- Название компании +
- З/п +
- Формат работы +
- Тип занятости +
- Навыки (Если можно достать отдельно) +
- Описание вакансии (если есть требования или навыки - то отлично, берем их) +
- Время публикации +
- Ссылка на вакансию тоже должна быть у нас +

In [97]:
id = 87774221



for key, value in res.items():
    print(key, value)

vac_link https://www.zarplata.ru/vacancy/card/id87774221
name Курьер
city Москва
company Рултек
experience Нет опыта
schedule Полный день
employment Полная занятость
skills [{'name': 'Мобильность'}, {'name': 'Пользователь ПК'}]
description <p><strong>Требуется курьер </strong>на доставку документов</p> <p> </p> <p><strong>Условия:</strong></p> <p>Доставка документов</p> <p>Проезд и мобильная связь оплачивается в полном размере.</p> <p>Зарплата выплачивается регулярно, 2 раза в месяц.</p>
salary {'from': 28000, 'to': 35000, 'currency': 'RUR', 'gross': False}
time 2023-10-05T12:08:57+0300
